Para ello ya escribimos un Dataset particular que lee el CSV y dado el índice de un ejemplo, calcula la posición de la cabeza, carga la imágen y retorna el par (imagen, posición) que para nuestro problema constituirán el par (x,y) (feature o variable independiente, target o variable dependiente):

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchvision.models
from torch import nn

import numpy as np
from matplotlib import pyplot
import math
import pickle
import gzip
from pathlib import Path
import requests
from tqdm import tqdm